In [2]:
# Pandas for data processing 
import pandas as pd


import numpy as np
from numpy import random

# Library to get countries codes
import pycountry

# Regex for string values
import re

     ---------------------------------------- 1.2/1.2 MB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 9.4/9.4 MB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 10.1/10.1 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 53.0/53.0 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 62.6/62.6 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 83.3/83.3 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 61.2/61.2 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 185.8/185.8 kB 2.3 MB/s eta 0:00:00
     ------------------------------------ 247.7/247.7 kB 724.0 kB/s eta 0:00:00
     ---------------------------------------- 48.0/48.0 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 98.0/98.0 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 

In [1]:
!pip install jupyterlab_github

     ---------------------------------------- 9.2/9.2 MB 11.5 MB/s eta 0:00:00
     ---------------------------------------- 66.0/66.0 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 57.2/57.2 kB 1.5 MB/s eta 0:00:00
     ------------------------------------- 377.6/377.6 kB 11.9 MB/s eta 0:00:00

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.1.5 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
conda 4.12.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.
selenium 4.5.0 requires urllib3[socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.



     -------------------------------------- 429.2/429.2 kB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 133.1/133.1 kB ? eta 0:00:00
     ---------------------------------------- 77.6/77.6 kB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 57.3/57.3 kB 2.9 MB/s eta 0:00:00
     -------------------------------------- 256.4/256.4 kB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 83.1/83.1 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 84.0/84.0 kB 4.6 MB/s eta 0:00:00
     -------------------------------------- 105.7/105.7 kB 3.1 MB/s eta 0:00:00


In [9]:
erasmus_df = pd.read_csv("Erasmus_mobility_statistics_2014_2019.csv",sep=',',low_memory=False)
erasmus_df=erasmus_df.dropna()
erasmus_df = erasmus_df.drop(columns = ['Project Reference','Sending Organisation Erasmus Code','Receiving Organisation Erasmus Code'])
erasmus_df

,Project Reference,Academic Year,Mobility Start Month,Mobility End Month,Mobility Duration,Activity (mob),Field of Education,Participant Nationality,Education Level,Participant Gender,...,Participant Age,Sending Country Code,Sending City,Sending Organization,Sending Organisation Erasmus Code,Receiving Country Code,Receiving City,Receiving Organization,Receiving Organisation Erasmus Code,Participants
0,2014-1-AT01-KA101-000059,2014-2015,2014-07,2014-07,12.0,Staff training abroad,Language acquisition,AT,??? - ? Unknown ?,Female,...,42,AT,Wels,BRG Wels Wallererstraße,-,ES,Malaga,Escuela de Idiomas Nerja,-,1.0
1,2014-1-AT01-KA101-000059,2014-2015,2014-07,2014-08,7.0,Staff training abroad,Language acquisition,AT,??? - ? Unknown ?,Female,...,25,AT,Wels,BRG Wels Wallererstraße,-,UK,Canterbury,Pilgrims LTD,-,1.0
2,2014-1-AT01-KA101-000059,2014-2015,2014-07,2014-08,12.0,Staff training abroad,Language acquisition,AT,??? - ? Unknown ?,Female,...,51,AT,Wels,BRG Wels Wallererstraße,-,UK,London,St. Giles International,-,1.0
3,2014-1-AT01-KA101-000059,2014-2015,2014-08,2014-08,12.0,Staff training abroad,Language acquisition,CZ,??? - ? Unknown ?,Female,...,34,AT,Wels,BRG Wels Wallererstraße,-,UK,London,EF Language Center,-,1.0
4,2014-1-AT01-KA101-000060,2014-2015,2014-07,2014-07,12.0,Staff training abroad,Teacher training with subject specialization,AT,??? - ? Unknown ?,Female,...,53,AT,Schwarzach,Volksschule Schwarzach,-,IE,Dublin,Centre of English Studies,-,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3462263,2019-2-UK01-KA105-062363,2019-2020,2019-09,2019-10,7.0,Mobility of youth workers - Programme Countries,? Unknown ?,TR,??? - ? Unknown ?,Female,...,24,UK,London,YOUTH INNOVATION LTD,-,UK,London,YOUTH INNOVATION LTD,-,1.0
3462264,2019-2-UK01-KA105-062363,2019-2020,2019-09,2019-10,7.0,Mobility of youth workers - Programme Countries,? Unknown ?,UK,??? - ? Unknown ?,Female,...,32,UK,London,YOUTH INNOVATION LTD,-,UK,London,YOUTH INNOVATION LTD,-,1.0
3462265,2019-2-UK01-KA105-062363,2019-2020,2019-09,2019-10,7.0,Mobility of youth workers - Programme Countries,? Unknown ?,UK,??? - ? Unknown ?,Female,...,37,UK,London,YOUTH INNOVATION LTD,-,UK,London,YOUTH INNOVATION LTD,-,1.0
3462266,2019-2-UK01-KA105-062363,2019-2020,2019-09,2019-10,7.0,Mobility of youth workers - Programme Countries,? Unknown ?,UK,??? - ? Unknown ?,Male,...,41,UK,London,YOUTH INNOVATION LTD,-,UK,London,YOUTH INNOVATION LTD,-,1.0


In [10]:
def drop_ages(x):
    if x > 90 or x < 10:
        return np.nan
    else:
        return x

In [11]:
##Supprimer les Staff et le Teaching et teaching
erasmus_df = erasmus_df[(~erasmus_df['Activity (mob)'].str.contains("Staff"))&(~erasmus_df['Activity (mob)'].str.contains("teaching"))&(~erasmus_df['Activity (mob)'].str.contains("Teaching"))]

In [3]:
erasmus_df['Participant Age'].sort_values(ascending=True)

NameError: name 'erasmus_df' is not defined